In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from scipy import stats
import math




# Import Sheet as csv 

In [2]:
ps = pd.read_csv('patch_seq_log_mouse.csv')

# Format sheet 

In [3]:
ps = ps[["Date", "User", "File", "Post patch?", "Notes", "Conc.", ">400", "Ratio", "SM_QC_PF", "Bad dates"]]
ps.dropna(subset = ["SM_QC_PF", "Post patch?"], inplace = True)
ps["User"] = ps["User"].astype("category")
ps["SM_QC_PF"] = ps["SM_QC_PF"].astype("category")
ps["Post patch?"] = ps["Post patch?"].astype("category")
ps["Conc."] = pd.to_numeric(ps["Conc."].str.replace(' ',''), errors ='force')
ps[">400"] = pd.to_numeric(ps[">400"].str.replace(' ',''), errors ='force')
ps["Date"] = pd.to_datetime(ps["Date"], format = '%y%m%d')
ps = ps[ps["SM_QC_PF"] != "cnt"]
ps = ps[ps["Bad dates"] != "x"]
ps.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
0,2016-03-18,P5,Ndnf - brl_160318_01,nr,NaN,106.83,NaN,NaN,fail,NaN
1,2016-03-18,P5,Ndnf - brl_160318_02,nr,Cell died at 6 mins,3649.03,NaN,NaN,fail,NaN
2,2016-03-18,P5,Ndnf - brl_160318_03,nr,NaN,70.89,NaN,NaN,fail,NaN
3,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.0,0.81,pass,NaN
4,2016-03-18,P5,Ndnf - brl_160318_05,nr,NaN,425.15,NaN,NaN,fail,NaN


In [4]:
ps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3359 entries, 0 to 6885
Data columns (total 10 columns):
Date           3359 non-null datetime64[ns]
User           3351 non-null category
File           3358 non-null object
Post patch?    3359 non-null category
Notes          2246 non-null object
Conc.          3246 non-null float64
>400           3311 non-null float64
Ratio          3228 non-null object
SM_QC_PF       3359 non-null category
Bad dates      127 non-null object
dtypes: category(3), datetime64[ns](1), float64(2), object(4)
memory usage: 220.0+ KB


# Percentage of nucleated patches passing SM_QC 

In [19]:
mask = ps["Post patch?"] == "Nucleated"
mask
nuc = ps[mask]
nuc.head()
nuc_count = nuc["Post patch?"].count()
nuc_count

920

In [9]:
ps["Post patch?"].value_counts()

No-Seal            921
Nucleated          920
Outside-Out        903
Partial-Nucleus    391
nr                 178
Entire-Cell         36
no-seal              5
outside-out          1
nucelated            1
no -seal             1
missing              1
Outside-Out?         1
Name: Post patch?, dtype: int64

In [10]:
pas = ps["SM_QC_PF"] == "pass"
pas.head()

0    False
1    False
2    False
3     True
4    False
Name: SM_QC_PF, dtype: bool

In [12]:
np = ps[mask & pas]
np

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
17,2016-04-06,P5,Ndnf - brl_160406_04,Nucleated,NaN,209.76,167.0,0.69,pass,NaN
29,2016-04-07,P1,Ndnf - kjh_160407_03,Nucleated,"cell swelled, no fluorescence visible in pipette",114.09,96.0,0.73,pass,NaN
52,2016-04-11,P5,Rorb - brl_160411_02,Nucleated,NaN,125.35,92.0,0.63,pass,NaN
69,2016-04-12,P5,Rorb - brl_160412_01,Nucleated,"couldnt see flourescence - cell too deep, but ...",195.16,183.0,0.87,pass,no slide in LIMS
109,2016-04-18,P5,Chat - brl_160418_02,Nucleated,"coudlnt get gigaseal, put still saw flur in pi...",135.75,116.0,0.85,pass,NaN
112,2016-04-18,P5,Chat - brl_160418_05,Nucleated,lost seal at 3 mins but still able to pull ou...,158.45,113.0,0.62,pass,NaN
125,2016-04-19,P1,Chat - kjh_160419_03,Nucleated,saw fluorescence in pipette,423.02,203.0,0.48,pass,NaN
144,2016-04-20,P1,Vip - kjh_160420_10,Nucleated,"saw fluorescence in pipette, cell dimmed",470.63,338.0,0.64,pass,NaN
188,2016-04-26,P5,Nos1 - brl_160426_02,Nucleated,NaN,353.39,282.0,0.73,pass,cell is between 01 and 02 (different than cell...
190,2016-04-26,P5,Nos1 - brl_160426_04,Nucleated,NaN,425.15,332.0,0.72,pass,NaN


In [13]:
nuc_pass = np["Post patch?"].count()
nuc_pass
#np["Post patch?"].count()

774

In [14]:
float(nuc_pass)/nuc_count

0.841304347826087

# Bubbles Analysis 

In [11]:
bubbles = ps.dropna(subset = ["Notes"])
bubbles.head()

,Date,User,File,Post patch?,Notes,Conc.,>400,Ratio,SM_QC_PF,Bad dates
3,2016-03-18,P5,Ndnf - brl_160318_04,Outside-Out,"broke in on its own, leaky",320.83,278.00,0.81,pass,NaN
16,2016-04-06,P5,Ndnf - brl_160406_03,Outside-Out,wiped with kim wipe without water before loadi...,100.61,37.65,NaN,fail,NaN
18,2016-04-06,P5,Ndnf - brl_160406_05,Partial-Nucleus,cell swelled,125.58,92.00,0.61,pass,NaN
29,2016-04-07,P1,Ndnf - kjh_160407_03,Nucleated patch,"cell swelled, no fluorescence visible in pipette",114.09,96.00,0.73,pass,NaN
30,2016-04-07,P1,Ndnf - kjh_160407_04,Partial-Nucleus,"cell swelled a lot, no fluorescence visible in...",129.23,111.00,0.79,pass,NaN


In [12]:
def make_mask(v,x):
    mask = v["Notes"].str.contains(x)
    df = v[mask]
    return df

def pass_only(y):
    mask = y["SM_QC_PF"] == "pass"
    df2 = y[mask]
    return df2

#Should also define a function to calculate pass percentages

In [13]:
def get_count(x):    
    count = x[">400"].count()
    return count

def get_sqrtct(x):
    sqrt_ct = math.sqrt(get_count(x))
    return sqrt_ct

def calc_mean(x):
    mean = x[">400"].mean()
    return mean

def calc_std(x):
    std_dev = x[">400"].std()
    return std_dev

def calc_stderr(x):
    std_err = calc_std(x) / get_sqrtct(x)
    return std_err

def calc_pp(passing,total):
    pp = float(get_count(passing)) / get_count(total)
    return pp

# No Bubbles Stats - All Samples

In [14]:
nb = make_mask(bubbles,"No Bubbles")

nb_mean = calc_mean(nb)
print "mean", nb_mean

nb_stderr = calc_stderr(nb)
print "standard error of the mean", nb_stderr

mean 1283.55656566
standard error of the mean 67.0597296871


# No Bubbles Stats - SMQC Pass Only 

In [15]:
nb_p = pass_only(make_mask(bubbles,"No Bubbles"))

nb_mean = calc_mean(nb_p)
print "mean =", nb_mean

nb_stderr = calc_stderr(nb_p)
print "standard error of the mean =", nb_stderr

nb_pp = calc_pp(nb_p,nb)
print "pass percentage =", nb_pp

mean = 1491.83888889
standard error of the mean = 71.847417749
pass percentage = 0.818181818182


# Small Bubbles Stats - All Samples 

In [16]:
sb = make_mask(bubbles,"Small Bubbles")

sb_mean = calc_mean(sb)
print "mean", sb_mean

sb_stderr = calc_stderr(sb)
print "standard error of the mean", sb_stderr

mean 1339.75222482
standard error of the mean 62.4661549106


# Small Bubbles Stats - SMQC Pass Only 

In [17]:
sb_p = pass_only(make_mask(bubbles,"Small Bubbles"))

sb_mean = calc_mean(sb_p)
print "mean", sb_mean

sb_stderr = calc_stderr(sb_p)
print "standard error of the mean", sb_stderr

sb_pp = calc_pp(sb_p,sb)
print "pass percentage =", sb_pp

mean 1624.07724551
standard error of the mean 72.4378510871
pass percentage = 0.782201405152


# Medium bubbles Stats - All Samples 

In [18]:
mb = make_mask(bubbles,"Medium Bubbles")

mb_mean = calc_mean(mb)
print "mean =", mb_mean

mb_stderr = calc_stderr(mb)
print "standard error of the mean =", mb_stderr

mean = 1315.29684211
standard error of the mean = 90.8850164303


# Medium Bubbles Stats - SMQC Pass Only

In [19]:
mb_p = pass_only(make_mask(bubbles, "Medium Bubbles"))

mb_mean = calc_mean(mb_p)
print "mean =", mb_mean

mb_stderr = calc_stderr(mb_p)
print "standard error of the mean =", calc_stderr(mb_p)

mb_pp = calc_pp(mb_p,mb)
print "pass percentage =", mb_pp

print get_count(mb)
print get_count(mb_p)

mean = 1545.98205128
standard error of the mean = 91.428673785
pass percentage = 0.821052631579
95
78


# Large Bubbles Stats - All Samples 

In [20]:
lb = make_mask(bubbles,"Large Bubbles")

lb_mean = calc_mean(lb)
print "mean =", lb_mean

lb_stderr = calc_stderr(lb)
print "standard error of the mean", lb_stderr

mean = 1367.83243243
standard error of the mean 193.073876424


# Large Bubbles Stats - SMQC Pass Only 

In [21]:
lb_p = pass_only(make_mask(bubbles,"Large Bubbles"))

lb_mean = calc_mean(lb_p)
print "mean =", lb_mean

lb_stderr = calc_stderr(lb_p)
print "standard error of the mean =", lb_stderr

lb_pp = calc_pp(lb_p,lb)
print "pass percentage =", lb_pp

mean = 1687.10689655
standard error of the mean = 209.797411073
pass percentage = 0.783783783784


# Solution up Pipette - All Samples 

In [22]:
sp = make_mask(bubbles,"Solution in Pipette Shank")

sp_mean = calc_mean(sp)
print "mean =", sp_mean

sp_stderr = calc_stderr(sp)
print "standard error of the mean =", sp_stderr

mean = 914.647368421
standard error of the mean = 295.180430139


# Solution up Pipette - SMQC Pass Only 

In [23]:
sp_p = pass_only(make_mask(bubbles, "Solution in Pipette Shank"))

sp_mean = calc_mean(sp_p)
print "mean =", sp_mean

sp_stderr = calc_stderr(sp_p)
print "standard error of the mean =", sp_stderr

sp_pp = calc_pp(sp_p,sp)
print "pass percentage =", sp_pp

print get_count(sp)
print get_count(sp_p)

mean = 1410.75454545
standard error of the mean = 458.825817884
pass percentage = 0.578947368421
19
11
